# Processing Data for Reproducible Analytics

## Explore the Data 

Begin by importing the required **Python** libraries and loading the **TZA_CCT_baseline.csv** dataset into Python. Following this, conduct a thorough inspection of the dataset to understand its structure and key characteristics. This inspection will be crucial for identifying the **Unit of Observation**, the **Unique ID**, the **Data Types**, and for gaining a rapid understanding of any potential **missing values** within the data.

In [1]:
## Import Libraries ##

# Pandas and Numpy for Data Manipulation, Analysis, and Numerical Calculations 
import pandas as pd
import numpy as np

# Matplotlib and Seaborn for Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Set Visualisation Theme
sns.set_theme()

## Load in the Data ##
df = pd.read_csv('Data/Raw/TZA_CCT_baseline.csv')

## Set the display option so that output display a maximum of 100 columns ##
pd.set_option('display.max_columns', 100)
## Explore the Data ##

# Display the structure of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   vid               1760 non-null   int64  
 1   hhid              1760 non-null   int64  
 2   enid              1760 non-null   int64  
 3   floor             1760 non-null   int64  
 4   roof              1760 non-null   int64  
 5   walls             1760 non-null   int64  
 6   water             1760 non-null   int64  
 7   enegry            1760 non-null   int64  
 8   rel_head          1760 non-null   int64  
 9   female_head       1760 non-null   int64  
 10  hh_size           1760 non-null   int64  
 11  n_child_5         1760 non-null   int64  
 12  n_child_17        1760 non-null   int64  
 13  n_adult           1760 non-null   int64  
 14  n_elder           1760 non-null   int64  
 15  read              1759 non-null   float64
 16  sick              1760 non-null   int64  


Analysis of the output reveals **1760** observations and **36** variables within the dataset. The data types include **27** integer (int64) columns, **4** string (object) columns, and **5** floating-point columns. The presence of missing values is evident in columns with fewer **non-null** entries. 

Examining the first **3** rows of the dataset will assist in determining the **unit of observation** and the **unique ID**.

In [2]:
# Display the first five rows of the dataset
print(df.head(3))

   vid  hhid  enid  floor  roof  walls  water  enegry  rel_head  female_head  \
0    1  9122   818      1     2      1      1       1         4            0   
1    1  9111   420      1     2      1      4       1         4            1   
2    1  9120   805      1     2      1      7       1         4            0   

   hh_size  n_child_5  n_child_17  n_adult  n_elder  read  sick  food_cons  \
0        4          0           1        1        2   0.0     1   595400.0   
1       10          1           3        3        3   0.0     0  1955200.0   
2        2          0           0        1        1   1.0     0   183820.0   

   nonfood_cons  farm   ar_farm ar_farm_unit  crop     crop_other  crop_prp  \
0         13600     1  0.500000         Acre  77.0            NaN       NaN   
1         69926     1  1.214083      Hectare  99.0  Coconut trees       2.0   
2          7500     1  0.250000         Acre  99.0         sesame       1.0   

   livestock_now  livestock_before  drought_flood

Analyzing the dataset's nature and structure, the **unit of observation** appears to be the **Household**. The variables consistently describe household characteristics, and each observation clearly corresponds to a single household. There is no evidence suggesting multiple units of observation.

Unique ID = **hhid**

## Identify and Fix Duplicates

In [3]:
# Identify duplicate by "hhid" column
duplicates = df['hhid'].duplicated().sum()

# Display the number of duplicates in the dataset
print(f"The Dataset has {duplicates} duplicates.")

The Dataset has 2 duplicates.


In [4]:
# Drop Duplicates -- the code will drop duplicates 
df.drop_duplicates(subset = ['hhid'], ignore_index = True, inplace = True)

# Check if indeed duplicates have been deleted 
new_duplicates = df['hhid'].duplicated().sum()
print(f"The new number of duplicated observation is: {new_duplicates}.")

The new number of duplicated observation is: 0.


## Clean the Data  

### Replace numeric values representing missing data (-88) with missings.

To address the presence of **-88** as a missing value indicator, these values will be replaced with **NaN**, the standard Python representation for missing data. Before applying the replacement, we will display the first five rows of a representative column containing **-88** to serve as a baseline for verification.

In [5]:
# Display the first five rows of "assoc" column
print(df['assoc'].head())

0     1
1   -88
2     0
3     0
4     0
Name: assoc, dtype: int64


In [6]:
# Replace numeric values representing missing data (-88) with NaN values.
df.replace(-88, np.nan, inplace = True)

# Display the "assoc" column again to check if the changes made are successful
print(df['assoc'].head())

0    1.0
1    NaN
2    0.0
3    0.0
4    0.0
Name: assoc, dtype: float64


The **-88** values have successfully been replaced with **NaN** as indicated in the `assoc` column snipet above. 

### Extend the values in `crop` column by adding the two most used categories from `crop_other`.

Before anything else, It is import to create a **Frequency Table** for `crop_other` column to check its **unique** values and their corresponding frequencies. 

In [7]:
# Creating frequency distribution table for crop_other" column
crop_other_freq = df['crop_other'].value_counts(dropna = False)

# Display the unique values
print(f"The unique values for crop_other are:\n {crop_other_freq}")

The unique values for crop_other are:
 crop_other
NaN               1659
sesame              31
Sesame              25
Sesame.             24
Coconut             15
Coconut.             2
Coconut trees        1
Coconut trees.       1
Name: count, dtype: int64


As evidenced by the table, this column exhibits inconsistent data entries. It contains a mix of intended missing value indicators, specific entries like **Seseme** and **Coconut**, and numerous typographical errors. Therefore, it is essential to clean this column before proceeding with any data extensions or modifications.

In [8]:
# Map old values to new values - Cleaning "crop_other" column
map = {'sesame': 'Sesame', 'Sesame.': 'Sesame', 'Sesame': 'Sesame', 'Coconut.':\
       'Coconut', 'Coconut trees': 'Coconut', 'Coconut trees.': 'Coconut', 'Coconut': 'Coconut'}

# Map Inconsistent values to correct values
df['crop_other'] = df['crop_other'].map(map)

# Display the new frequency table to check if the column has been cleaned
crop_other_freq = df['crop_other'].value_counts(dropna = False)
print(f"The new frequency table for crop_other is:\n {crop_other_freq}")

The new frequency table for crop_other is:
 crop_other
NaN        1659
Sesame       80
Coconut      19
Name: count, dtype: int64


The `crop_other` column is now cleaned and ready for further analysis. The next step is to augment the `crop` column by incorporating the two most common entries from `crop_other`. To better understand the interplay between these columns, the following is a table snippet that specifically highlights instances where `crop_other` contains **Sesame** or **Coconut**.

In [9]:
df_crop = df[df['crop_other'].isin(['Sesame', 'Coconut'])][['crop', 'crop_other']]
print(df_crop.sample(10, random_state = 45))

      crop crop_other
1     99.0    Coconut
2     99.0     Sesame
1095  99.0     Sesame
1261  99.0     Sesame
666   99.0     Sesame
1749  99.0    Coconut
1266  99.0     Sesame
668   99.0     Sesame
66    99.0     Sesame
1401  99.0    Coconut


Given that the `crop` column is intended as a numerical representation of the categorical values in `crop_other`, it's crucial that each unique category in `crop_other` has a unique numerical code in `crop`. To integrate 'Sesame' and 'Coconut' from crop_other into crop, it makes sense to assign them two previously unused numerical codes.

In [10]:
### Extend the values in crop column by adding the two most used categories from crop_other
df['crop'] = df['crop'].mask(df['crop_other'] == 'Sesame', 40)
df['crop'] = df['crop'].mask(df['crop_other'] == 'Coconut', 41)

# Verify if the values have indeed been changed
df_crop = df[df['crop_other'].isin(['Sesame', 'Coconut'])][['crop', 'crop_other']]
print(df_crop.sample(10, random_state = 45))

      crop crop_other
1     41.0    Coconut
2     40.0     Sesame
1095  40.0     Sesame
1261  40.0     Sesame
666   40.0     Sesame
1749  41.0    Coconut
1266  40.0     Sesame
668   40.0     Sesame
66    40.0     Sesame
1401  41.0    Coconut


In [11]:
# Display unique values for crop column to verify if "99" is realy gone
crop_unique = df['crop'].unique()

# Display the unique values
print(f"\nThe Unique values for 'crop' column are:\n {crop_unique}")


The Unique values for 'crop' column are:
 [77. 41. 40.  6.  4.  9.  5. 34.  1. 15. nan 11. 32. 39.  3.  8. 21. 20.
 24. 18. 10. 17. 31.  2. 13.]


`crop` column and `crop_other` columns have successfully been cleaned.

### Check and document if there are numeric variable that have outliers

The following activities will check for otliers in **Consumption** and **Area variables**.

**Consumption Varaibles**: `food_cons` and `nonfood_cons`

**Area Variables**: `ar_farm`

The first step is to generate the summary statistics for the three columns.

In [12]:
df_stats = df[['food_cons', 'nonfood_cons', 'ar_farm']].describe()
print(f"\nSummary Statistics for 'Consumption' and 'Area' variables: \n\n {df_stats}")


Summary Statistics for 'Consumption' and 'Area' variables: 

           food_cons  nonfood_cons      ar_farm
count  1.758000e+03  1.758000e+03  1620.000000
mean   6.392233e+05  1.468731e+05     1.321132
std    5.131717e+05  2.510197e+05     1.061838
min    5.096000e+03  0.000000e+00     0.125000
25%    2.616250e+05  2.347750e+04     1.000000
50%    5.174000e+05  6.993300e+04     1.214083
75%    8.580000e+05  1.665750e+05     2.000000
max    4.903600e+06  4.108400e+06    32.375556


The summary statistics will facilitate the calculation of **interquartile** ranges (IQRs). These **IQR**s will subsequently be used to define **upper** and **lower** outlier **thresholds**. Data points that fall beyond these thresholds will be flagged as **outliers**.

#### Check for outliers in `food_cons` column

In [13]:
# Determine the first Quartile (Q1) and Third Quartile (Q3)
Q1 = df_stats.loc['25%', 'food_cons']
Q3 = df_stats.loc['75%', 'food_cons']

# Calculate "Interquatile
IQR = Q3 - Q1

# Define the multiplier
multiplier = 1.5

# Calculate "Upper" and "Lower" Thresholds using IQR and multiplier
upper_threshold = Q3 + (IQR * multiplier)
lower_threshold = Q1 - (IQR * multiplier)

# Find Outliers
outliers = df[(df['food_cons'] > upper_threshold) | (df['food_cons'] < lower_threshold)]['food_cons']

# Display Outliers
print(f"\n The Outliers for 'food_cons' columns are:\n {outliers}")


 The Outliers for 'food_cons' columns are:
 1       1955200.0
6       1842360.0
9       1903200.0
24      2002000.0
35      2243800.0
          ...    
1562    2405000.0
1598    1801800.0
1607    2246400.0
1626    1874600.0
1633    2150200.0
Name: food_cons, Length: 65, dtype: float64


The output indicates that the column `food_cons` has **65** outliers.

#### Check for outliers in `nonfood_cons` column

In [14]:
# Determine the first Quartile (Q1) and Third Quartile (Q3)
Q1 = df_stats.loc['25%', 'nonfood_cons']
Q3 = df_stats.loc['75%', 'nonfood_cons']

# Calculate "Interquatile
IQR = Q3 - Q1

# Define the multiplier
multiplier = 1.5

# Calculate "Upper" and "Lower" Thresholds using IQR and multiplier
upper_threshold = Q3 + (IQR * multiplier)
lower_threshold = Q1 - (IQR * multiplier)

# Find Outliers
outliers = df[(df['nonfood_cons'] > upper_threshold) | (df['nonfood_cons'] < lower_threshold)]['nonfood_cons']

# Display Outliers
print(f"\nThe Outliers for 'nonfood_cons' column are: \n {outliers}")


The Outliers for 'nonfood_cons' column are: 
 9       553500
22      567900
32      417400
41      396300
48      630600
         ...  
1685    576300
1714    386800
1719    454800
1720    445700
1723    641400
Name: nonfood_cons, Length: 157, dtype: int64


`nonfood` column has **157** outliers.

#### Check for outliers in `ar_farm` column

In [15]:
# Determine the first Quartile (Q1) and Third Quartile (Q3)
Q1 = df_stats.loc['25%', 'ar_farm']
Q3 = df_stats.loc['75%', 'ar_farm']

# Calculate "Interquatile
IQR = Q3 - Q1

# Define the multiplier
multiplier = 1.5

# Calculate "Upper" and "Lower" Thresholds using IQR and multiplier
upper_threshold = Q3 + (IQR * multiplier)
lower_threshold = Q1 - (IQR * multiplier)

# Find Outliers
outliers = df[(df['ar_farm'] > upper_threshold) | (df['ar_farm'] < lower_threshold)]['ar_farm']

# Display Outliers
print(f"\nThe Outliers for 'ar_farm' column are:\n {outliers}")


The Outliers for 'ar_farm' column are:
 132      4.046945
183      6.070417
386      4.856333
499      5.261028
639      3.642250
656      4.046945
673      6.475111
745      4.046945
998      4.046945
1001     6.475111
1003     3.642250
1006     6.879806
1147     4.046945
1153     4.451639
1271     4.451639
1345    32.375556
1453     6.070417
1497     4.046945
1503     4.046945
1607     3.642250
1609     4.046945
Name: ar_farm, dtype: float64


`ar_farm`column has **21** outliers.

## Saving the cleaned Dataset

The cleaned dataset will be saved in Data/Intermediate folder. 

In [16]:
df.to_csv("Data/Intermediate/TZA_CCT_baseline_new.csv", encoding = "utf8", index = False)

# Preparing Data for Reproducible Analytics